## TensorBoard With PyTorch

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120) # Display options for o/p
torch.set_grad_enabled(True) # Already on by default

from torch.utils.tensorboard import SummaryWriter

In [2]:
print(torch.__version__)
print(torchvision.__version__)

1.8.1
0.9.1


In [3]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [4]:
class Network(nn.Module):

    def __init__(self):

        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):

        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = F.relu(self.fc1(t.reshape(-1, 12*4*4)))
        t = F.relu(self.fc2(t))
        t = self.out(t)

        return t

In [11]:
train_set = torchvision.datasets.FashionMNIST(
    root = './FashionMNIST',
    train=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ]),
    download=True
)

In [6]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)

## Starting out with TensorBoard (Network Graph and Images)

In [7]:
tb = SummaryWriter()

network = Network()
images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images) # to make a grid of images

tb.add_graph(network, images) # adding the model and this grid on the TensorBoard

tb.close() # close the summary writer

## Training with multiple epochs: The complete training loop

In [8]:
network = Network()

optimizer = optim.Adam(network.parameters(), lr=0.01)

for epoch in range(1):

    total_loss = 0
    total_correct = 0

    for batch in train_loader: # Get Batch
        images, labels = batch

        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad() # zeros the gradients (as gradients are accumulated)
        
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print(f"epoch: {epoch} | total_correct: {total_correct} | loss: {total_loss}")

epoch: 0 | total_correct: 46965 | loss: 344.8538021147251


## Tensorboard in Action

In [9]:
network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images) # to make a grid of images

tb = SummaryWriter()
tb.add_image('image', grid)
tb.add_graph(network, images) # adding the model and this grid on the TensorBoard


for epoch in range(10):

    total_loss = 0
    total_correct = 0

    for batch in train_loader: # Get Batch
        images, labels = batch

        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad() # zeros the gradients (as gradients are accumulated)
        
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)
        
    # Adding these values in TB
    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Number Correct', total_correct, epoch)
    tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)
    
    tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
    tb.add_histogram('conv1.weight', network.conv1.weight, epoch)
    tb.add_histogram('conv1.weight.grad', network.conv1.weight.grad, epoch)

    print(f"epoch: {epoch} | total_correct: {total_correct} | loss: {total_loss}")
    
tb.close() # close the summary writer

epoch: 0 | total_correct: 47247 | loss: 333.8133105635643
epoch: 1 | total_correct: 51556 | loss: 227.7684179842472
epoch: 2 | total_correct: 52228 | loss: 210.24352623522282
epoch: 3 | total_correct: 52397 | loss: 205.39225906133652
epoch: 4 | total_correct: 52639 | loss: 199.94638223946095
epoch: 5 | total_correct: 52925 | loss: 191.98648872971535
epoch: 6 | total_correct: 53145 | loss: 185.2769401818514
epoch: 7 | total_correct: 53186 | loss: 185.85071274638176
epoch: 8 | total_correct: 53349 | loss: 182.13601101189852
epoch: 9 | total_correct: 53426 | loss: 177.99371188879013


In [10]:
total_correct / len(train_set)

0.8904333333333333

__View results in Tensorboard <br>
Run this command in cmd__ <br>
<br>
<code> tensorboard --logdir=runs </code>